In [28]:
from bs4 import BeautifulSoup
from ebooklib import epub
import re
import datetime

import requests
response = requests.get('https://mtg.gamepedia.com/Magic_Story')
html_doc = response.text

soup = BeautifulSoup(html_doc, 'html.parser')

In [16]:
def getDate(string):
    match = re.search(r'\d{4}-\d{2}-\d{2}', string)
    date = datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
    return date

In [17]:
links = list()

for link in soup.find_all('a'):
    links.append(str(link.get('href')))
    
links_filtered = [x for x in links if 'http://magic.wizards.com/' in x]
links_sorted = links_filtered
#links_filtered



In [18]:
links_sorted.append('http://magic.wizards.com/en/articles/archive/magic-story/favor-2017-07-05')
links_sorted.append('http://magic.wizards.com/en/articles/archive/magic-story/hour-eternity-2017-07-12')
links_sorted.append('http://magic.wizards.com/en/articles/archive/magic-story/endure-2017-07-19')
links_sorted.append('http://magic.wizards.com/en/articles/archive/magic-story/hour-devastation-2017-07-26')

In [19]:
links_sorted[links_sorted.index('http://magic.wizards.com/en/articles/archive/magic-story/sbreaking-points-2017-01-18')] = 'http://magic.wizards.com/en/articles/archive/magic-story/breaking-points-2017-01-18'


In [20]:
def makeTitlePretty(soupTitle):
    stringTitle = str(soupTitle)
    stringTitle = stringTitle.split('>')[1].split('|')[0]
    return stringTitle

In [21]:
def createChapter(url):
    response = requests.get(url)
    temp_doc = response.text
    sub_soup = BeautifulSoup(temp_doc, 'html.parser')
    c1_title = str(makeTitlePretty(sub_soup.title))

    content = sub_soup.find_all("div", id="content-detail-page-of-an-article")
    c1_content = str(content[0].find_all('body')).replace('[','').replace(']','')
    return c1_title, c1_content

# Gesamtcrawler

In [22]:
chapters = dict()   
for link in links_sorted:
    c_title, c_content = createChapter(link)
    print(c_title + ' done.')
    chapters[c_title] = c_content
    


Odric, Master Tactician  done.
Xathrid Gorgon  done.
Chronomaton  done.
Krenko, Mob Boss  done.
Threadbare  done.
Talrand, Sky Summoner  done.
The Stonekiller  done.
The Stonekiller, Part 2  done.
The Shadows of Prahv, Part 1  done.
The Shadows of Prahv, Part 2  done.
Epic Experiment  done.
In Praise of the Worldsoul, Part 1  done.
In Praise of the Worldsoul, Part 2  done.
In Praise of the Worldsoul, Part 3  done.
Slaughter Games  done.
The Great Concourse  done.
The Azorius Ten Most Wanted  done.
The Seven Bells, Part 1  done.
The Seven Bells, Part 2  done.
Rogue's Passage  done.
Gruul Ingenuity  done.
The Fathom Edict  done.
The Absolution of the Guildpact  done.
Persistence of Memory  done.
The Burying, Part 1  done.
The Greater Good  done.
The Guild of Deals  done.
Experiment One  done.
Fblthp  done.
The Burying, Part 2  done.
Bilagru Will Come for You  done.
The Hard Sell  done.
Behind the Black Sun  done.
Ruric Thar  done.
Teysa Karlov  done.
Barrin's Tall Tale  done.
Expectation

# Create Epub

In [23]:
def create_and_add_chapter(c_title, c_content, book):
    
    c_xhtml = c_title+'.xhtml'

    chapter = epub.EpubHtml(title=c_title, file_name=c_xhtml, lang='hr')
    chapter.content=u'<h1>'+ c_title + '</h1>' + str(c_content)

    ### add chapter
    magic_stories_epub.add_item(chapter)
    return chapter, book

In [24]:
magic_stories_epub = epub.EpubBook()
magic_stories_epub.set_title('Magic the Gathering Stories') 
magic_stories_epub.set_language('en')

In [25]:
ebook_chapters = list()

for c_title in chapters:
    chap, magic_stories_epub = create_and_add_chapter(c_title, chapters[c_title], magic_stories_epub)
    ebook_chapters.append(chap)


In [26]:
### define Table Of Contents
magic_stories_epub.toc = (epub.Link('Test1', 'Test2', 'Test3'),(epub.Section('Stories'),ebook_chapters))

### add default NCX and Nav file
magic_stories_epub.add_item(epub.EpubNcx())
magic_stories_epub.add_item(epub.EpubNav())

### define CSS style
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

### add CSS file
magic_stories_epub.add_item(nav_css)

### basic spine
spine = ['nav']
spine.extend(ebook_chapters)
magic_stories_epub.spine = spine

### write to the file
epub.write_epub('test.epub', magic_stories_epub, {})

In [27]:
len(links_sorted)

246

## Notes

http://magic.wizards.com/en/articles/archive -  Alle Artikel

http://magic.wizards.com/en/articles/columns/magic-story - Story Auflistungsseite


from ebooklib import epub

magic_stories_epub = epub.EpubBook()

### set metadata
book.set_identifier('id123456')
book.set_title('Sample book')
book.set_language('en')

book.add_author('Author Authorowski')
book.add_author('Danko Bananko', file_as='Gospodin Danko Bananko', role='ill', uid='coauthor')

### create chapter
c1 = epub.EpubHtml(title='Intro', file_name='chap_01.xhtml', lang='hr')
c1.content=u'<h1>Intro heading</h1><p>Žaba je skočila u baru.</p>'

### add chapter
book.add_item(c1)

### define Table Of Contents
book.toc = (epub.Link('chap_01.xhtml', 'Introduction', 'intro'),
(epub.Section('Simple book'),
(c1, ))
)

### add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

### define CSS style
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

### add CSS file
book.add_item(nav_css)

### basic spine
book.spine = ['nav', c1]

### write to the file
epub.write_epub('test.epub', book, {})